# ResNet18 as a feature extractor




In [ ]:
%matplotlib inline

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, \
    accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from IPython.display import clear_output

plt.ion()   # interactive mode

#Mounting Google Drive data
from google.colab import drive
drive.mount('/content/gdrive')

#Use GPU device
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Used device:", device)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Used device: cuda


ConvNet as fixed feature extractor


Feature vectors are extracted from the second last layer of the ResNet18 (vector of 512 elements)




The ResNet18 used is the pretrained one on ImageNet

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import csv

from os import walk

# Root directory of interest
gdrivePath = F"gdrive/MyDrive/ML_AI" # where to save accuracies, losses and model checkpoints (csv)

# Dataset to use
data_dir = gdrivePath + "/NatureDataset"

# Load the pretrained model
model = models.resnet18(pretrained=True) #github code was with ResNet18

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
model.eval()

# Image transforms
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(image_name, classID):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)

    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
 
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    #my_embedding = torch.zeros(512)
    my_embedding = torch.zeros(1, 512, 1, 1) #CORRECTED ACCORDING TO: https://github.com/josharnoldjosh/Resnet-Extract-Image-Feature-Pytorch-Python/issues/1

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    #  for resnet18:
    feature_vector = my_embedding.numpy()[0, :, 0, 0] #TO RETRIEVE ONLY THE 512 ELEMENTS   
    
    feature_vector = np.insert(feature_vector, 0, classID)  #insert the class ID in the first position
    return feature_vector

train_path = data_dir + "/train"
test_path = data_dir + "/test"
fv_list = []

classes_dict = {
  "buildings": 0,
  "forest": 1,
  "glacier": 2,
  "mountain": 3,
  "sea": 4,
  "street": 5
}

path_to_use = test_path
for (_, dirnames, filenames) in walk(path_to_use):
    dirnames = sorted(dirnames)
    for classDir in dirnames:
        classID = classes_dict[classDir] #retrieve the class id         
        print("Extracting features from "+classDir+"...")

        for (_, dirnames, filenames) in walk(path_to_use+"/"+classDir):
            
            #print(filenames)
            for fn in filenames:
                fv = get_vector(path_to_use + "/" + classDir + "/" + fn, classID)
                fv_list.append(fv)
    break

#Write the feature vectors on a .csv file
with open('feature_vectors_test.csv', 'w') as f:      
    write = csv.writer(f)      
    write.writerows(fv_list)
